In [1]:
import pandas as pd
from sqlite3 import connect
import xlwings as xw
import numpy as np
from datetime import datetime as dt
import os
import pathlib
import sources.parts_list_0802 as pl

def working_day(start_date,end_date):
    try:
        res = np.busday_count(start_date.strftime('%Y-%m-%d'),
                          start_date.strftime('%Y-%m-%d'))
    except:
        res = np.busday_count(start_date,end_date)
    return res + 1 
# conn.create_function("WORKING_DAY",2,working_day)

conn = connect('history.db')

q = '''SELECT * FROM new_ml WHERE status  IS not "0] CLOSED"'''
new_ml = pd.read_sql(q,conn)
rma_list = tuple(new_ml['ExFM RMA#'])
print(len(rma_list),max(rma_list))

307 FMSV2024010037


In [2]:
#priority receive, inspect, quote, confirm, part order, part receive
# new format combine with ML
# filter RMA 

q = f'''
        SELECT DISTINCT c.[rma no.] as [ExFM RMA#],
        CASE 
            WHEN m.receive IS NOT NULL THEN strftime('%Y-%m-%d',m.receive)
            ELSE strftime('%Y-%m-%d',recieve_date) END as [Receive],
        strftime('%Y-%m-%d',R.[start time]) as [Start Date],
        strftime('%Y-%m-%d',c.qc) as [End Date],
        '' as [Repair TAT],
        repair_status AS [Repair Status],c.approval,
        CASE
            WHEN repair_status IS 'Receive' THEN '1] RECEIVE'
            WHEN repair_status IS 'Inspection' THEN '2] INSPECTION'
            WHEN repair_status IS 'Parts Selection' THEN '3] PARTS SELECTION'
            WHEN repair_status IS 'Authorization'  AND c.approval IS 'Approval' 
                AND c.[Status Info] IS 'PO Received' THEN '4C] Approved Repair'
            
            WHEN repair_status IS 'Authorization'  AND r.[start time] IS NOT NULL THEN '5] REPAIR'
            WHEN repair_status IS 'Authorization'  AND c.approval IS NOT 'Approval' THEN '4B] Decline Repair'
            WHEN repair_status IS 'Authorization'  AND c.approval IS 'Approval' 
                AND (c.[Status Info] IS 'Waiting for Parts' OR c.[Status Info] IS 'Parts on Order') 
                THEN '4A] Waiting for Parts'

            WHEN repair_status IS 'Repair' THEN '5] REPAIR'
            WHEN repair_status IS 'QC' THEN '6] QC'
            WHEN repair_status IS 'Shipped' THEN '7] SHIPPED'
            WHEN repair_status IS 'Completed' THEN '0] CLOSED'
            
            ELSE repair_status END AS STATUS,
        c.[status info],strftime('%Y-%m-%d',c.[Due Date]) AS [Due Date],
        e.short_name AS Repairer,
        CASE
            WHEN m.location IS NOT NULL THEN m.location
            WHEN c.ship_user_name IS NOT NULL THEN ship.location
            WHEN c.[repair user name] IS NOT NULL THEN repair.location
            WHEN c.in_inspect_user_name IS NOT NULL THEN inspect.location
            ELSE (receive.location) END AS location, 
        
        customer_name AS [Customer Name],c.model,UPPER(serial_no) AS SN,c.[Date Installed],
        m.[information from customer],m.[New Status],
        [defect note] AS [Problem Descript.],
        CASE
            WHEN c.approval IN ('Decline','Cancel','Transfer') THEN 'Return'
            ELSE c.[repair size] END AS repair_size,
        m.issue AS Issue,
        CASE 
            WHEN m.[inspect date] IS NOT NULL THEN strftime('%Y-%m-%d',m.[inspect date])
            ELSE strftime('%Y-%m-%d',c.in_inspect_date) END AS [Inspect Date],
        CASE 
            WHEN m.[quote date] IS NOT NULL THEN m.[quote date]
            ELSE strftime('%Y-%m-%d',[Part Select Date]) END AS [Quote Date],
        CASE
            WHEN m.[confirm date] IS NOT NULL THEN m.[confirm date]
            ELSE strftime('%Y-%m-%d',AUTHORIZED_DATE) END AS [Confirm Date],
        '' AS [PO TAT],
        m.[Part Order Date] AS [Part Order Date],
        m.[Part Receive Date] AS [Part Receive Date],
        '' [Parts TAT],
        '' AS [Total TAT],
        strftime('%Y-%m-%d',c.shipped) AS Shipped,c.[other id],c.[internal note],c.[Loaner Shp Note],
        CASE
            WHEN m.[repair size] IN ("Minor",'Other') THEN 200
            WHEN issue LIKE "%dry%" THEN 600
            WHEN issue LIKE "%cha%" OR issue LIKE "%LGB%" THEN 500
            WHEN issue LIKE "%FCT%" OR issue LIKE "%AWT%" OR issue LIKE "%DEC%" THEN 450
            WHEN issue LIKE "%BSA%" OR issue LIKE "%FSA" OR issue LIKE "%FSB" THEN 400
            ELSE 350 END AS [Service_Fee],
        m.[Quote Price (VND)],install.[group]
        
        FROM (((((((consolidated c
        LEFT JOIN repair_code r ON c.[rma no.] = r.[rma no.])
        LEFT JOIN new_ml m ON c.[rma no.] = m.[exfm rma#])
        
        LEFT JOIN engineers e ON c.[repair user name] = e.exfm_name)
        LEFT JOIN engineers receive ON c.RECIEVE_USER_NAME = receive.exfm_name)
        LEFT JOIN engineers inspect ON c.in_inspect_user_name = inspect.exfm_name)
        LEFT JOIN engineers repair ON c.[repair user name] = repair.exfm_name)
        LEFT JOIN engineers ship ON c.ship_user_name = ship.exfm_name)
        LEFT JOIN install ON c.serial_no = install.[serial no.]
        
        WHERE 
            c.[rma no.] IN {rma_list}
            OR c.[rma no.] > '{max(rma_list)}'
        
        ORDER BY 1
    '''
            
        
try:
    hcap = pd.read_sql(q,conn)
    display(hcap)
except Exception as e:
    print(e)

,ExFM RMA#,Receive,Start Date,End Date,Repair TAT,Repair Status,Approval,STATUS,Status Info,Due Date,...,Part Receive Date,Parts TAT,Total TAT,Shipped,Other ID,Internal Note,Loaner Shp Note,Service_Fee,Quote Price (VND),Group
0,FMSV2023030069,2023-03-28,None,None,,Authorization,Approval,4A] Waiting for Parts,Parts on Order,None,...,None,,,None,PO116,None,SGA1,500,226000000,SCOPE
1,FMSV2023030079,2023-03-30,None,None,,Parts Selection,None,3] PARTS SELECTION,Awating Contract billing,2024-01-30,...,None,,,None,None,None,None,200,268000000,PROCESSOR
2,FMSV2023040070,2023-04-26,None,None,,Authorization,Approval,4C] Approved Repair,PO Received,None,...,2023-12-27 00:00:00,,,None,PO115,None,SGA2,500,None,SCOPE
3,FMSV2023050045L,2023-06-13,None,None,,Authorization,Approval,4C] Approved Repair,PO Received,None,...,2023-12-27 00:00:00,,,None,PO115,Hoan My Sai Gon\r\ndrying by heating and vacuu...,SGA1,600,None,SCOPE
4,FMSV2023050072,2023-05-29,None,None,,Authorization,Approval,4A] Waiting for Parts,Parts on Order,None,...,None,,,None,None,QC after loaner HNNA\r\nBlack spot's at 7o'clo...,None,600,None,SCOPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,FMSV2024010043,2024-01-23,2024-01-26,None,,Authorization,Approval,5] REPAIR,None,None,...,None,,,None,None,None,None,350,None,SCOPE
313,FMSV2024010044,2024-01-23,None,None,,Parts Selection,None,3] PARTS SELECTION,None,None,...,None,,,None,None,None,None,350,None,SCOPE
314,FMSV2024010045,2024-01-25,2024-01-26,None,,Authorization,Approval,5] REPAIR,None,None,...,None,,,None,None,None,None,350,None,SCOPE
315,FMSV2024010046,None,None,None,,Create,None,Create,None,None,...,None,,,None,None,None,None,350,None,SCOPE


In [3]:
#calculate TAT

for i in range(len(hcap)):
    rma = hcap['ExFM RMA#'][i]
    issue = pl.summary_report().issues(conn,rma)
    # TAT repair
    if hcap['Start Date'][i] is not None and hcap['End Date'][i] is not None:
        hcap['Repair TAT'][i] = working_day(hcap['Start Date'][i],hcap['End Date'][i])
    # PO TAT
    if hcap['Quote Date'][i] is not None and hcap['Confirm Date'][i] is not None:
        try:
            hcap['PO TAT'][i] = working_day(hcap['Quote Date'][i],hcap['Confirm Date'][i])
        except:
#             print(f"{hcap['SN'][i]} failed PO TAT")
            pass
    # issue
    if hcap['Issue'][i] is None:
        hcap['Issue'][i] = issue
    if hcap['Repair Status'][i] != 'Completed':
        try:
            hcap['Total TAT'] = working_day(hcap['Receive'][i],dt.now().strftime('%Y-%m-%d'))
        except Exception as e:
            print(e,f"{hcap['SN'][i]}")
    else:
        if hcap['Shipped'][i] is not None:
            hcap['Total TAT'] = working_day(hcap['Receive'][i],hcap['Shipped'][i])
hcap

Cannot compute a business day count with a NaT (not-a-time) date 1G391K869


,ExFM RMA#,Receive,Start Date,End Date,Repair TAT,Repair Status,Approval,STATUS,Status Info,Due Date,...,Part Receive Date,Parts TAT,Total TAT,Shipped,Other ID,Internal Note,Loaner Shp Note,Service_Fee,Quote Price (VND),Group
0,FMSV2023030069,2023-03-28,None,None,,Authorization,Approval,4A] Waiting for Parts,Parts on Order,None,...,None,,1,None,PO116,None,SGA1,500,226000000,SCOPE
1,FMSV2023030079,2023-03-30,None,None,,Parts Selection,None,3] PARTS SELECTION,Awating Contract billing,2024-01-30,...,None,,1,None,None,None,None,200,268000000,PROCESSOR
2,FMSV2023040070,2023-04-26,None,None,,Authorization,Approval,4C] Approved Repair,PO Received,None,...,2023-12-27 00:00:00,,1,None,PO115,None,SGA2,500,None,SCOPE
3,FMSV2023050045L,2023-06-13,None,None,,Authorization,Approval,4C] Approved Repair,PO Received,None,...,2023-12-27 00:00:00,,1,None,PO115,Hoan My Sai Gon\r\ndrying by heating and vacuu...,SGA1,600,None,SCOPE
4,FMSV2023050072,2023-05-29,None,None,,Authorization,Approval,4A] Waiting for Parts,Parts on Order,None,...,None,,1,None,None,QC after loaner HNNA\r\nBlack spot's at 7o'clo...,None,600,None,SCOPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,FMSV2024010043,2024-01-23,2024-01-26,None,,Authorization,Approval,5] REPAIR,None,None,...,None,,1,None,None,None,None,350,None,SCOPE
313,FMSV2024010044,2024-01-23,None,None,,Parts Selection,None,3] PARTS SELECTION,None,None,...,None,,1,None,None,None,None,350,None,SCOPE
314,FMSV2024010045,2024-01-25,2024-01-26,None,,Authorization,Approval,5] REPAIR,None,None,...,None,,1,None,None,None,None,350,None,SCOPE
315,FMSV2024010046,None,None,None,,Create,None,Create,None,None,...,None,,1,None,None,None,None,350,None,SCOPE


In [5]:
today = dt.now().strftime('%y%m%d')
# today = '231019'
folder_name = 'inspection'
path = pathlib.Path().absolute()
file_name = f'FFVN-Repair_{today}.xlsx'
file_name = os.path.join(path,folder_name,file_name)
# hcap.to_excel(file_name,index = False)

with pd.ExcelWriter(file_name,date_format="YYYY-MM-DD",
    datetime_format="YYYY-MM-DD HH:MM:SS") as writer:
    hcap.to_excel(writer,index = False)

#OPEN FILE 
xw.Book(file_name)

<Book [FFVN-Repair_240129.xlsx]>

### fy2023 1h


In [6]:
hcap.to_sql('hcap',conn,index=False,if_exists='replace')

317

### Receive


In [9]:
q = f'''
        SELECT [exfm rma#],[customer name],hcap.model,sn,receive,
        CASE
            WHEN c.ship_user_name IS NOT NULL THEN ship.location
            WHEN c.[repair user name] IS NOT NULL THEN repair.location
            WHEN c.in_inspect_user_name IS NOT NULL THEN inspect.location
            ELSE (receive.location) END AS location
        
        FROM ((((hcap
        LEFT JOIN consolidated c ON hcap.[exfm rma#] = c.[rma no.])
        LEFT JOIN engineers receive ON c.RECIEVE_USER_NAME = receive.exfm_name)
        LEFT JOIN engineers inspect ON c.in_inspect_user_name = inspect.exfm_name)
        LEFT JOIN engineers repair ON c.[repair user name] = repair.exfm_name)
        LEFT JOIN engineers ship ON c.ship_user_name = ship.exfm_name
        
        WHERE [exfm rma#] > '{max(rma_list)}'
    '''
print(max(rma_list))
new_receive = pd.read_sql(q,conn)
new_receive

FMSV2024010037


,ExFM RMA#,Customer Name,MODEL,SN,Receive,location
0,FMSV2024010038,Vietnam – Cuba Donghoi Friendship Hospital,EG-760R,KG402K677,2024-01-23,Hanoi
1,FMSV2024010039,"FUJIFILM Vietnam Co., Ltd.",EG-530WR,4G361K141,2024-01-23,Hanoi
2,FMSV2024010040,"FUJIFILM Vietnam Co., Ltd.",EPX-2500(230V),5V564D396,2024-01-23,Hanoi
3,FMSV2024010041,Hai Phong International Hospital,EG-250WR5,2G202K044,2024-01-23,Hanoi
4,FMSV2024010042,Hai Phong International Hospital,EG-250WR5,2G202K043,2024-01-23,Hanoi
5,FMSV2024010043,"FUJIFILM Vietnam Co., Ltd.",EC-530WI,1C603K041,2024-01-23,Hanoi
6,FMSV2024010044,Tra Vinh General Hospital,EC-250WL5,3C309K056,2024-01-23,HCM
7,FMSV2024010045,"FUJIFILM Vietnam Co., Ltd.",EC-600WI,1C692K249,2024-01-25,Hanoi
8,FMSV2024010046,Bach Mai Hospital,EG-600WR,1G391K869,None,None
9,FMSV2024010047,Hue University Hospital of Medicine And Pharmacy,EC-530WI3,2C642D166,2024-01-29,HCM


### Complete

In [ ]:
# tracking
q= 'SELECT DISTINCT STATUS FROM new_ml'
pd.read_sql(q,conn)

In [8]:
# tracking
q = '''
        SELECT [ExFM RMA#] FROM new_ml 
        WHERE status IN ('1] RECEIVE','2] INSPECTION','3] PARTS SELECTION',
                        '4A] Waiting for Parts','4C] Approved Repair','5] REPAIR')
    '''
pending = pd.read_sql(q,conn)

#completed
q = f'''
        SELECT DISTINCT [customer name],model,sn,location,repair_size,issue,status,[end date],shipped,[exfm rma#]
        
        FROM hcap
        WHERE ([exfm rma#] IN {tuple(pending['ExFM RMA#'])}
        OR [exfm rma#] IN {tuple(new_receive['ExFM RMA#'])})
        AND status IN ('0] CLOSED','4B] Declined Repair','6] QC','7] SHIPPED')
        ORDER BY status
    '''
completed = pd.read_sql(q,conn)
# completed.to_excel(f'inspection/completed_{today}.xlsx',index=False)
completed

,Customer Name,MODEL,SN,location,repair_size,Issue,STATUS,End Date,Shipped,ExFM RMA#
0,Dam Doi Hospital,EG-250WR5,SG202A469,HCM,Return,"LGB, FSA, FSB, PCB",0] CLOSED,None,2024-01-24,FMSV2023090073
1,"FUJIFILM Vietnam Co., Ltd.",EC-590WL4,1C666K485,HCM,Return,"OSA, VCA",0] CLOSED,None,2024-01-23,FMSV2024010004
2,"FUJIFILM Vietnam Co., Ltd.",EG-720R,JG402K330,Hanoi,Minor,CLN,0] CLOSED,2024-01-22,2024-01-22,FMSV2024010033
3,"FUJIFILM Vietnam Co., Ltd.",EC-720R/I,1C741K089,Hanoi,Return,"CHA, FSB",0] CLOSED,None,2024-01-23,FMSV2024010034
4,"FUJIFILM Vietnam Co., Ltd.",EP-6000,1V696K004,Hanoi,Other,O,0] CLOSED,2024-01-22,2024-01-22,FMSV2024010035
5,"FUJIFILM Vietnam Co., Ltd.",EG-600WR,1G391K524,HCM,Major,BSA,6] QC,2024-01-24,None,FMSV2023080038
6,Hoang Long Clinic,VP-4450HD,1V567K251,Hanoi,Return,PCB,6] QC,None,None,FMSV2023110009
7,Hoang Long Clinic,EG-600WR,1G391K374,Hanoi,Return,"FCT, CHA, PCB, LGB, BSA",6] QC,None,None,FMSV2023110045
8,"FUJIFILM Vietnam Co., Ltd.",SU-1-H,3V658K014,Hanoi,Other,CLN,6] QC,2024-01-26,None,FMSV2024010010
9,"FUJIFILM Vietnam Co., Ltd.",EN-450T5,1C675D055,Hanoi,Major,FSA,6] QC,2024-01-26,None,FMSV2024010011


### Analys

In [ ]:
conn = connect('history.db')

###### 1. Receive

In [ ]:
q = '''
        SELECT [exfm rma#] ,[customer name],[model],[sn],location
        FROM new_ml
        WHERE status = '1] RECEIVE'
    '''
pd.read_sql(q,conn)

###### 2. Inspection

In [ ]:
q = '''
        SELECT [exfm rma#] ,[customer name],[model],[sn],location,
        [status info],[due date],[internal note],[quote price (vnd)] AS Price
        FROM new_ml
        WHERE status = '2] INSPECTION'
        AND Price IS NOT NULL
    '''
pd.read_sql(q,conn)

###### 3. Part Selection

In [ ]:
# No Price
q = '''
        SELECT [exfm rma#] ,[customer name],[model],[sn],location,
        [status info],[information from cusomter],[due date],[internal note],[quote price (vnd)] AS Price
        FROM new_ml
        WHERE status = '3] PARTS SELECTION'
        AND PRICE IS NULL
    '''
pd.read_sql(q,conn)

###### Activity this week

In [ ]:
week_day = dt.now().weekday()

In [ ]:
from datetime import datetime, timedelta

N_DAYS_AGO = 5

today = datetime.now()    
monday = today - timedelta(days=week_day)
print (monday)

In [ ]:
type(monday)

In [ ]:
monday.strftime('%Y-%m-%d')

In [ ]:
aa

In [ ]:
# monday = ''
monday = monday.strftime('%Y-%m-%d')
q = f'''
        SELECT * 
        
        FROM consolidated
        WHERE [UPDATE TIME] >'{monday}'
        ORDER BY [update time]
    '''
activity_this_week = pd.read_sql(q,conn)
activity_this_week.to_sql('thisweek',conn,if_exists = 'replace')

In [ ]:
# REceive
q = f'''
        SELECT [rma no.] ,[customer_name],[model],[serial_no],
        strftime('%Y-%m-%d',recieve_date) AS [Receive Date],
        CASE
            WHEN c.ship_user_name IS NOT NULL THEN ship.location
            WHEN c.[repair user name] IS NOT NULL THEN repair.location
            WHEN c.in_inspect_user_name IS NOT NULL THEN inspect.location
            ELSE (receive.location) END AS location
        
        From ((((thisweek c
        LEFT JOIN engineers e ON c.[repair user name] = e.exfm_name)
        LEFT JOIN engineers receive ON c.RECIEVE_USER_NAME = receive.exfm_name)
        LEFT JOIN engineers inspect ON c.in_inspect_user_name = inspect.exfm_name)
        LEFT JOIN engineers repair ON c.[repair user name] = repair.exfm_name)
        LEFT JOIN engineers ship ON c.ship_user_name = ship.exfm_name
        where recieve_date > '{monday}'
    '''
pd.read_sql(q,conn)

In [ ]:
# Inspection fix location
q = f'''
        SELECT [rma no.] ,[customer_name],[model],[serial_no],
        strftime('%Y-%m-%d',recieve_date) AS [Receive Date],
        strftime('%Y-%m-%d',in_inspect_date) AS [Inspect Date],
        in_inspect_user_name as [Inspector],
        
        CASE
            WHEN c.ship_user_name IS NOT NULL THEN ship.location
            WHEN c.[repair user name] IS NOT NULL THEN repair.location
            WHEN c.in_inspect_user_name IS NOT NULL THEN inspect.location
            ELSE (receive.location) END AS location
        From ((((thisweek c
        LEFT JOIN engineers e ON c.[repair user name] = e.exfm_name)
        LEFT JOIN engineers receive ON c.RECIEVE_USER_NAME = receive.exfm_name)
        LEFT JOIN engineers inspect ON c.in_inspect_user_name = inspect.exfm_name)
        LEFT JOIN engineers repair ON c.[repair user name] = repair.exfm_name)
        LEFT JOIN engineers ship ON c.ship_user_name = ship.exfm_name
        where in_inspect_date > '{monday}'
    '''
pd.read_sql(q,conn)

In [ ]:
# Sent Quote/Approval
q = f'''
        SELECT [rma no.] ,[customer_name],[model],[serial_no],
        strftime('%Y-%m-%d',in_inspect_date) AS [Inspect Date],
        strftime('%Y-%m-%d',[Part Select Date]) AS [Quote Date],
        
        CASE
            WHEN c.ship_user_name IS NOT NULL THEN ship.location
            WHEN c.[repair user name] IS NOT NULL THEN repair.location
            WHEN c.in_inspect_user_name IS NOT NULL THEN inspect.location
            ELSE (receive.location) END AS location
        From ((((thisweek c
        LEFT JOIN engineers e ON c.[repair user name] = e.exfm_name)
        LEFT JOIN engineers receive ON c.RECIEVE_USER_NAME = receive.exfm_name)
        LEFT JOIN engineers inspect ON c.in_inspect_user_name = inspect.exfm_name)
        LEFT JOIN engineers repair ON c.[repair user name] = repair.exfm_name)
        LEFT JOIN engineers ship ON c.ship_user_name = ship.exfm_name
        where [Part Select Date] > '{monday}'
    '''
pd.read_sql(q,conn)

In [ ]:
# Get Confirmation
q = f'''
        SELECT [rma no.] ,[customer_name],[model],[serial_no],
        strftime('%Y-%m-%d',[Part Select Date]) AS [Quote Date],
        strftime('%Y-%m-%d',[AUTHORIZED_DATE]) AS [Confirm Date],
        [other id],
        
        CASE
            WHEN c.ship_user_name IS NOT NULL THEN ship.location
            WHEN c.[repair user name] IS NOT NULL THEN repair.location
            WHEN c.in_inspect_user_name IS NOT NULL THEN inspect.location
            ELSE (receive.location) END AS location
        From ((((thisweek c
        LEFT JOIN engineers e ON c.[repair user name] = e.exfm_name)
        LEFT JOIN engineers receive ON c.RECIEVE_USER_NAME = receive.exfm_name)
        LEFT JOIN engineers inspect ON c.in_inspect_user_name = inspect.exfm_name)
        LEFT JOIN engineers repair ON c.[repair user name] = repair.exfm_name)
        LEFT JOIN engineers ship ON c.ship_user_name = ship.exfm_name
        where [AUTHORIZED_DATE] > '{monday}'
    '''
pd.read_sql(q,conn)

In [ ]:
# Repair
q = f'''
        SELECT DISTINCT c.[rma no.] ,[customer_name],[model],[serial_no],
        strftime('%Y-%m-%d',[AUTHORIZED_DATE]) AS [Confirm Date],
        strftime('%Y-%m-%d',r.[start time]) AS [Start Date],
        strftime('%Y-%m-%d',c.[qc]) AS [End Date],
        CASE
            WHEN r.[end user] IS NOT NULL THEN r.[end user]
            ELSE r.[start user] END AS Repairer,
        
        [other id],
        
        CASE
            WHEN c.ship_user_name IS NOT NULL THEN ship.location
            WHEN c.[repair user name] IS NOT NULL THEN repair.location
            WHEN c.in_inspect_user_name IS NOT NULL THEN inspect.location
            ELSE (receive.location) END AS location
        From (((((thisweek c
        LEFT JOIN repair_code r ON c.[rma no.] = r.[rma no.])
        LEFT JOIN engineers e ON c.[repair user name] = e.exfm_name)
        LEFT JOIN engineers receive ON c.RECIEVE_USER_NAME = receive.exfm_name)
        LEFT JOIN engineers inspect ON c.in_inspect_user_name = inspect.exfm_name)
        LEFT JOIN engineers repair ON c.[repair user name] = repair.exfm_name)
        LEFT JOIN engineers ship ON c.ship_user_name = ship.exfm_name
        
        where r.[start time] > '{monday}'
    '''
pd.read_sql(q,conn)

In [ ]:
# Ship out
q = f'''
        SELECT DISTINCT c.[rma no.] ,[customer_name],[model],[serial_no],
        strftime('%Y-%m-%d',[recieve_date]) AS [Receive Date],
        strftime('%Y-%m-%d',[in_inspect_date]) AS [Inspect Date],
        strftime('%Y-%m-%d',c.[qc]) AS [End Date],
        strftime('%Y-%m-%d',c.[shipped]) AS [Ship Date],
        CASE
            WHEN r.[end user] IS NOT NULL THEN r.[end user]
            ELSE r.[start user] END AS Repairer,
        
        [other id],
        
        CASE
            WHEN c.ship_user_name IS NOT NULL THEN ship.location
            WHEN c.[repair user name] IS NOT NULL THEN repair.location
            WHEN c.in_inspect_user_name IS NOT NULL THEN inspect.location
            ELSE (receive.location) END AS location,
        repair_status
        From (((((thisweek c
        LEFT JOIN repair_code r ON c.[rma no.] = r.[rma no.])
        LEFT JOIN engineers e ON c.[repair user name] = e.exfm_name)
        LEFT JOIN engineers receive ON c.RECIEVE_USER_NAME = receive.exfm_name)
        LEFT JOIN engineers inspect ON c.in_inspect_user_name = inspect.exfm_name)
        LEFT JOIN engineers repair ON c.[repair user name] = repair.exfm_name)
        LEFT JOIN engineers ship ON c.ship_user_name = ship.exfm_name
        
        WHERE c.shipped > '{monday}'
        ORDER BY [ship date]
    '''
pd.read_sql(q,conn)